In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import JavascriptException, NoSuchElementException, StaleElementReferenceException
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
from selenium.common.exceptions import ElementNotInteractableException, ElementNotSelectableException
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time
from IPython.display import clear_output
import os
import base64
from github import Github
from github import InputGitTreeElement
from datetime import datetime

In [6]:
download_path = 'C:\\Users\\rlitt\\Documents\\GitHub\\perry'
chrome_options = Options()
#chrome_options.add_argument("--headless")
chrome_options.add_argument("--window-size=1920x1080")
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--verbose')
chrome_options.add_experimental_option("prefs", {
        "download.default_directory": download_path,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing_for_trusted_sources_enabled": False,
        "safebrowsing.enabled": False
})
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-software-rasterizer')
driver = webdriver.Chrome(options=chrome_options, executable_path="chromedriver.exe")
now = datetime.now()

In [7]:
def get_new_pdf():
    #load page
    print('getting pdf')
    link_to_results_pdf = '//*[@id="main_content"]/div/div[1]/div/div[2]/a[1]'
    driver.get('https://perryco.org/departments/elections-voter-registration/')
    time.sleep(5)
    #wait for link to load
    #driver.wait(link_to_results_pdf)
    #assign link to pdf
    pdf_url = driver.find_element_by_xpath(link_to_results_pdf).get_attribute('href')
    driver.get(pdf_url)

def delete_pdf():
    os.remove(f'{download_path}\\county election results.pdf')
    print('deleting old version')

#cribbed from https://stackoverflow.com/questions/50071841/how-to-push-local-files-to-github-using-python-or-post-a-commit-via-python
def commit_new_pdf():
    print('pushing pdf')
    user = "ryanelittle"
    password = ""
    g = Github(user,password)
    repo = g.get_user().get_repo('perry')
    file_list = [
        f'{download_path}\\county election results.pdf'
    ]

    file_names = [
        'perry-election-results.pdf'
    ]
    commit_message = f'refreshed update at {now.strftime("%H:%M:%S")}'
    master_ref = repo.get_git_ref('heads/master')
    master_sha = master_ref.object.sha
    base_tree = repo.get_git_tree(master_sha)
    element_list = list()
    for i, entry in enumerate(file_list):
        with open(entry) as input_file:
        if entry.endswith('.png'):
            data = base64.b64encode(data)
        element = InputGitTreeElement(file_names[i], '100644', 'blob', data)
        element_list.append(element)
    tree = repo.create_git_tree(element_list, base_tree)
    parent = repo.get_git_commit(master_sha)
    commit = repo.create_git_commit(commit_message, tree, [parent])
    master_ref.edit(commit.sha)
    print(f'pdf pushed at {now.strftime("%H:%M:%S")}')

In [8]:
count = 0
for i in range(10000000):
    count = count + 1
    print(f'Starting loop {str(count)}.')
    get_new_pdf()
    time.sleep(15)
    try:
        commit_new_pdf()
    except:
        pass
    time.sleep(15)
    delete_pdf()
    time.sleep(30)
    clear_output(wait=True)

Starting loop 597.
getting pdf
pushing pdf


FileNotFoundError: [WinError 2] The system cannot find the file specified: 'C:\\Users\\rlitt\\Documents\\GitHub\\perry\\county election results.pdf'

In [24]:
get_new_pdf()

getting pdf


In [25]:
commit_new_pdf()

pushing pdf
pdf pushed at 18:13:28


In [27]:
user = "ryanelittle"
password = "g = Github(user,password)
repo = g.get_user().get_repo('perry')
file_list = [
    f'{download_path}\\county election results.pdf'
]

file_names = [
    'perry-election-results.pdf'
]
commit_message = f'refreshed update at {now.strftime("%H:%M:%S")}'
master_ref = repo.get_git_ref('heads/master')
master_sha = master_ref.object.sha
base_tree = repo.get_git_tree(master_sha)
element_list = list()
for i, entry in enumerate(file_list):
    with open(entry) as input_file:
        data = input_file.read()
    if entry.endswith('.png'):
        data = base64.b64encode(data)
    element = InputGitTreeElement(file_names[i], '100644', 'blob', data)
    element_list.append(element)
tree = repo.create_git_tree(element_list, base_tree)
parent = repo.get_git_commit(master_sha)
commit = repo.create_git_commit(commit_message, tree, [parent])
master_ref.edit(commit.sha)